In [11]:
import numpy as np
import pandas as pd
import requests
import cPickle as pickle
from bs4 import BeautifulSoup
import time
from time import sleep
import re

In [151]:
#check openfile pickle, then read

df = pd.read_pickle("./data.pickle")


In [152]:
df.head(1)

,company,country,description,location,review,salary,star,title
10,UMeKang,Singapore,"Use all available data, undertaking data disco...",Singapore,None,"$9,000 - $11,000 a month",None,Data Scientist


In [153]:
df.reset_index(drop=True, inplace=True)

In [154]:
df.head(1)

,company,country,description,location,review,salary,star,title
0,UMeKang,Singapore,"Use all available data, undertaking data disco...",Singapore,None,"$9,000 - $11,000 a month",None,Data Scientist


In [155]:
df[df.duplicated('description') == True].shape

(304, 8)

In [156]:
df.drop_duplicates('description',keep='first', inplace=True)

In [157]:
df.head(1)

,company,country,description,location,review,salary,star,title
0,UMeKang,Singapore,"Use all available data, undertaking data disco...",Singapore,None,"$9,000 - $11,000 a month",None,Data Scientist


In [158]:
df.shape
df_us = df[df.country == 'United States']

In [141]:
df.loc[df[df.location.str.contains('TX')].index.values,'location'] = 'Texas'
df.loc[df[df.location.str.contains('NY')].index.values,'location'] = 'New York'
df.loc[df[df.location.str.contains('WA')].index.values,'location'] = 'Washington'
df.loc[df[df.location.str.contains('CA')].index.values,'location'] = 'California'
df.loc[df[df.location.str.contains('IL')].index.values,'location'] = 'Illinois'
df.loc[df[df.location.str.contains('MD')].index.values,'location'] = 'Maryland'
df.loc[df[df.location.str.contains('PA')].index.values,'location'] = 'Pennsylvania'
df.loc[df[df.location.str.contains('GA')].index.values,'location'] = 'Georgia'
df.loc[df[df.location.str.contains('NJ')].index.values,'location'] = 'New Jersey'
df.loc[df[df.location.str.contains('WI')].index.values,'location'] = 'Wisconsin'

In [159]:
df.shape

(4039, 8)

In [160]:
df_us['period'] = df_us.salary.map(lambda x: x.split()[-1])

df_us.period = df_us.period.map(lambda x: 12 if x == 'month' else 52 if x == 'week' else
                                260 if x == 'day' else 2080 if x == 'hour' else 1)

regex = re.compile(r'[\d\-\.]+')
df_us.salary = df_us.salary.map(lambda x: ''.join(regex.findall(x)))

df_us.salary = [float(i) if i.replace('.','').isdigit() == True else np.mean([float(i.split('-')[0]),float(i.split('-')[1])]) 
                 for i in df_us.salary.values]

df_us.salary = [row[-1] * row[-4] for row in df_us.itertuples()]

/Users/Naren/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [161]:
#df = df_us

In [162]:
df_us.head(1)

,company,country,description,location,review,salary,star,title,period
2923,Crossover,United States,Ignite's FirstRain SaaS-based analytics soluti...,Remote,11,100000.0,30.0,Data Scientist,1


## Question1

In [163]:
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [165]:
#dummy location dataframe from location so that it is 1/0

location_df = pd.get_dummies(df.location, drop_first = True, prefix='city/state')
country_df = pd.get_dummies(df.country, drop_first = True, prefix = 'ctry')
period_df = pd.get_dummies(df.period, drop_first = True, prefix='period')


In [166]:
X = pd.concat([location_df,country_df,period_df], axis = 1)
y = df.salary

In [167]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 8)

In [168]:
enetcv = ElasticNetCV(n_alphas=100, l1_ratio=np.linspace(0.01, 1.0, 25), cv = 10, n_jobs=-1, verbose = 1)
enetcv.fit(X_train,y_train)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
       l1_ratio=array([ 0.01   ,  0.05125,  0.0925 ,  0.13375,  0.175  ,  0.21625,
        0.2575 ,  0.29875,  0.34   ,  0.38125,  0.4225 ,  0.46375,
        0.505  ,  0.54625,  0.5875 ,  0.62875,  0.67   ,  0.71125,
        0.7525 ,  0.79375,  0.835  ,  0.87625,  0.9175 ,  0.95875,  1.     ]),
       max_iter=1000, n_alphas=100, n_jobs=-1, normalize=False,
       positive=False, precompute='auto', random_state=None,
       selection='cyclic', tol=0.0001, verbose=1)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [169]:
enet = ElasticNet(alpha=enetcv.alpha_, l1_ratio=enetcv.l1_ratio_, random_state = 8)
enet.fit(X_train, y_train)

ElasticNet(alpha=19.747610927717517, copy_X=True, fit_intercept=True,
      l1_ratio=1.0, max_iter=1000, normalize=False, positive=False,
      precompute=False, random_state=8, selection='cyclic', tol=0.0001,
      warm_start=False)

In [170]:
enet_score = enet.score(X_test,y_test)
enetcv_score = np.mean(cross_val_score(enet, X, y, cv = 5))

In [171]:
#randomforest regression
rr = RandomForestRegressor(n_estimators=1000, max_depth=5, n_jobs=-1)
rr.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=-1, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [172]:
rr_score = rr.score(X_test,y_test)
rrcv_score = np.mean(cross_val_score(rr, X, y, cv = 5))

In [173]:
# gridsearch params
rr_params = {
    'max_depth':[1,2,3,4,5],
    'max_features':[1,2,3,4,5],
}

# set the gridsearch
rr_gs = GridSearchCV(RandomForestRegressor(n_estimators=1000), rr_params, cv=5, verbose=1, n_jobs=-1)

In [174]:
rr_gs.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   39.1s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  2.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_features': [1, 2, 3, 4, 5], 'max_depth': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [175]:
print rr_gs.best_params_
print rr_gs.best_score_

{'max_features': 5, 'max_depth': 5}
0.0255411297598


In [176]:
rr_best = rr_gs.best_estimator_
rr_best_score = rr_best.score(X_test,y_test)
rrcv_best_score = np.mean(cross_val_score(rr_best,X,y,cv = 5))

In [177]:
pd.DataFrame({'score':[enet_score, rr_score, rr_best_score],'cv_score':[enetcv_score,rrcv_score,rrcv_best_score]},
            index = ['enet','randomforestreg','randomforestreg_gs'])

,cv_score,score
enet,0.110080,0.250587
randomforestreg,0.081942,0.169371
randomforestreg_gs,0.008752,0.058507


In [180]:
#df.salary_class.value_counts()

In [178]:
df['salary_class'] = df.salary.map(lambda x: 'low' if x < 90000 else 'high')

In [179]:
df.salary_class = df.salary_class.map(lambda x: 1 if x == 'high' else 0)

In [181]:
X = pd.concat([location_df,country_df,period_df], axis = 1)
y = df.salary_class

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [182]:
svc = SVC()
svc.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [183]:
df.salary_class.value_counts()

0    568
1    213
Name: salary_class, dtype: int64

In [184]:
baseline_score = float(np.sum(y))/len(y)

In [185]:
yhat = svc.predict(X_test)

In [186]:
svc_score = svc.score(X_test, y_test)
svc_auc = roc_auc_score(y_test,yhat)
svc_f1 = f1_score(y_test, yhat)

/Users/Naren/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [187]:
svc_params = {
    'C':np.logspace(-2,3,10),
    'gamma':np.logspace(-5,-2,20),
    'kernel':['rbf']
}

In [189]:
#set paramaters
svc_gs = GridSearchCV(svc,svc_params,n_jobs = -1,cv = 5, verbose =1)

In [190]:
svc_gs.fit(X_train,y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   12.6s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'kernel': ['rbf'], 'C': array([  1.00000e-02,   3.59381e-02,   1.29155e-01,   4.64159e-01,
         1.66810e+00,   5.99484e+00,   2.15443e+01,   7.74264e+01,
         2.78256e+02,   1.00000e+03]), 'gamma': array([  1.00000e-05,   1.43845e-05,   2.06914e-05,   2.97635e-05,
         4.2813...,   1.62378e-03,   2.33572e-03,
         3.35982e-03,   4.83293e-03,   6.95193e-03,   1.00000e-02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [191]:
svc_best=SVC(C = 1000, gamma = 0.01, kernel = 'rbf')
svc_best.fit(X_train,y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [192]:
yhat = svc_best.predict(X_test)

In [193]:
svc_best_score = svc_best.score(X_test,y_test)
svc_best_auc = roc_auc_score(y_test,yhat)
svc_best_f1 = f1_score(y_test,yhat)

In [ ]:
#confusion matrix

In [194]:
print classification_report(y_test,yhat, labels = [0,1], target_names=['low','high'])

print '--------------------------------------------------'

print pd.DataFrame(confusion_matrix(y_test,yhat,labels=[0,1]), index = ['is_low','is_high'], columns = ['pred_low','pred_high'])

             precision    recall  f1-score   support

        low       0.79      0.95      0.86       193
       high       0.62      0.25      0.35        65

avg / total       0.75      0.77      0.73       258

--------------------------------------------------
         pred_low  pred_high
is_low        183         10
is_high        49         16


In [195]:
#set rand forest params
randoclass = RandomForestClassifier(n_estimators=1000,max_depth=10, n_jobs=-1)
randoclass.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [197]:
randoclass.score(X_test,y_test)

0.7558139534883721

In [198]:
fi = pd.DataFrame({'features':X.columns, 'importance':randoclass.feature_importances_})
fi.sort_values('importance', ascending = False).head(10)

,features,importance
197,period_12,0.075492
93,"city/state_Los Angeles County, CA",0.074316
159,"city/state_San Francisco, CA",0.067458
200,period_2080,0.056543
168,"city/state_Santa Monica, CA",0.042414
67,"city/state_Houston, TX",0.034644
113,"city/state_New York, NY",0.029580
82,"city/state_Irving, TX",0.027674
34,"city/state_Chicago, IL",0.027672
15,"city/state_Austin, TX 78759 (Arboretum area)",0.026795


by the above models, the top factors in determining the job are the features given above in the decreasing rate of importance.